In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U bitsandbytes # This ensures to install latest version of bitsandbytes
!pip install -q transformers accelerate bitsandbytes huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

In [ ]:
# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load the base model with quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# Load  LoRA adapter weights directly from your Drive path
adapter_path = "/content/drive/MyDrive/Assignment/2025/T2/Hallucination detection/1.3 fine tune/medical_lora_adapter"
model = PeftModel.from_pretrained(base_model, adapter_path)

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Function to generate answers
def generate_medical_answer(question):
    # Format the question with the prompt template
    prompt = f"<s>[INST] <<SYS>>\nYou are a helpful, respectful and Medical assistant.\n<</SYS>>\n\nQuestion: answer this question : {question} [/INST]"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        do_sample=True,
        temperature=0.5,
        top_p=1,
        top_k=60,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the model's response part (after [/INST])
    response_only = response.split("[/INST]")[-1].strip()
    return response_only

In [ ]:
# Testing again with sample question
sample_question = " I have knee pain which medicine should i take?"
print("\n Testing with revised prompt for sample question:", sample_question)
sample_answer = generate_medical_answer(sample_question)
sample_answer


 Testing with revised prompt for sample question:  I have knee pain which medicine should i take?


'Answer: {\'contexts\': [\'The aim of the study was to compare the effectiveness between two different types of medication for osteoarthritis in patients with chronic knee pain (CKP).\', \'A total of 103 CKP subjects were randomly assigned into three groups receiving either diclofenac sodium tablet (DS), diclofenac sodium gel (DSG) or placebo; all treatments lasted four weeks. The Western Ontario McMaster Universities Osteoarthritis Index (WOMAC) score, visual analog scale (VAS) pain score, WOMAC stiffness score and patient global assessment (PGA) score were evaluated before treatment and after one week\'s follow-up period.\', "There was no significant difference among DSG, DS group and control group on VAS pain scores at baseline level. After one week\'s follow up, there is significantly lowering in VAS pain scores compared to that at baseline levels only in the DSG group."], \'labels\': [\'OBJECTIVE\', \'METHOD\', \'RESULT\'], \'meshes\': [\'Adolescent\','

# test model

In [ ]:
# 0) Install + imports
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer, util
import torch, pandas as pd

In [ ]:
# 1) Gold Q&A set
data = [
    {"question": "What is the chemical symbol for water?",
     "standard_answer": "H₂O"},
    {"question": "Which planet is known as the Red Planet?",
     "standard_answer": "Mars"},
    {"question": "What is the capital city of France?",
     "standard_answer": "Paris"},
    {"question": "How many continents are there on Earth?",
     "standard_answer": "Seven (Africa, Antarctica, Asia, Europe, North America, Oceania, South America)"},
    {"question": "Who wrote the play “Romeo and Juliet”?",
     "standard_answer": "William Shakespeare"},
    {"question": "What is the largest mammal in the world?",
     "standard_answer": "The blue whale (Balaenoptera musculus)"},
    {"question": "In what year did the Titanic sink?",
     "standard_answer": "1912"},
    {"question": "What is the value of π (pi) to two decimal places?",
     "standard_answer": "3.14"},
    {"question": "Which element has the atomic number 6?",
     "standard_answer": "Carbon"},
    {"question": "What language is primarily spoken in Brazil?",
     "standard_answer": "Portuguese"},
]

In [ ]:







device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()                 # uses your existing `model`
embedder = SentenceTransformer("all-MiniLM-L6-v2").to(device)

# 3) Sampling grid
N = 5
seeds = [42, 43, 44, 45, 46][:N]
temps = [0.5, 0.6, 0.7, 0.8, 0.9][:N]
settings = list(zip(seeds, temps))

def gen_answer(prompt, seed, temp):
    torch.manual_seed(seed)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=temp,
            top_p=0.9,
            top_k=50,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # keep only the generated completion after "Answer:" if present
    if "Answer:" in text:
        text = text.split("Answer:", 1)[1].strip()
    return text

rows = []
overall_scores = []

for item in data:
    q, gold = item["question"], item["standard_answer"]

    # 4) Generate multiple samples
    samples = [gen_answer(f"Question: {q}\nAnswer:", s, t) for s, t in settings]

    # 5) Embed gold + samples
    gold_emb   = embedder.encode(gold,    convert_to_tensor=True, device=device)
    sample_emb = embedder.encode(samples, convert_to_tensor=True, device=device)

    # 6) Cosine similarities
    sims = util.cos_sim(gold_emb, sample_emb)[0].tolist()
    avg_score = sum(sims) / len(sims)
    overall_scores.append(avg_score)

    # 7) Collect for table
    for (seed, temp), sim, ans in zip(settings, sims, samples):
        rows.append({
            "question": q,
            "seed": seed,
            "temp": temp,
            "similarity": round(sim, 3),
            "generated_answer": ans,
            "gold_answer": gold
        })






modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 8) Summary + pretty table
df = pd.DataFrame(rows)
display(df)

per_q = df.groupby("question")["similarity"].mean().reset_index(name="avg_similarity")
display(per_q)

overall_avg = sum(overall_scores) / len(overall_scores)
print(f"\n=== Overall Average Semantic Similarity across {len(per_q)} questions: {overall_avg:.3f} ===")

,question,seed,temp,similarity,generated_answer,gold_answer
0,What is the chemical symbol for water?,42,0.5,0.134,H2O.\nQuestion: What is the chemical symbol fo...,H₂O
1,What is the chemical symbol for water?,43,0.6,0.269,H2O.\nWater is a chemical compound with the ch...,H₂O
2,What is the chemical symbol for water?,44,0.7,0.340,H2O\nHydrogen and oxygen are the two most comm...,H₂O
3,What is the chemical symbol for water?,45,0.8,0.128,H2O\nQuestion: What is the chemical symbol for...,H₂O
4,What is the chemical symbol for water?,46,0.9,0.192,H2O.\nQuestion: What is the symbol for the num...,H₂O
5,Which planet is known as the Red Planet?,42,0.5,0.496,Mars\nThe planet Mars is known as the Red Plan...,Mars
6,Which planet is known as the Red Planet?,43,0.6,0.418,Mars\nQuestion: Which planet is known as the B...,Mars
7,Which planet is known as the Red Planet?,44,0.7,0.397,Mars\nQuestion: The name of the god of the sea...,Mars
8,Which planet is known as the Red Planet?,45,0.8,0.463,Mars is called the Red Planet because of the r...,Mars
9,Which planet is known as the Red Planet?,46,0.9,0.330,Mars\nQuestion: How long is the life of an ave...,Mars


,question,avg_similarity
0,How many continents are there on Earth?,0.5354
1,In what year did the Titanic sink?,0.3244
2,What is the capital city of France?,0.4538
3,What is the chemical symbol for water?,0.2126
4,What is the largest mammal in the world?,0.4214
5,What is the value of π (pi) to two decimal pla...,0.2836
6,What language is primarily spoken in Brazil?,0.5894
7,Which element has the atomic number 6?,0.4850
8,Which planet is known as the Red Planet?,0.4208
9,Who wrote the play “Romeo and Juliet”?,0.5090



=== Overall Average Semantic Similarity across 10 questions: 0.424 ===


In [ ]:
# # simple flag if below threshold (tune threshold for your domain, e.g., 0.80)
# threshold = 0.80
# flags = per_q[per_q["avg_similarity"] < threshold]
# if len(flags):
#     print("\nQuestions potentially at risk of inconsistency (below threshold):")
#     for _, r in flags.iterrows():
#         print(f"- {r['question']}  avg_similarity={r['avg_similarity']:.3f}")
# else:
#     print("\nAll questions meet the similarity threshold.")

# function

In [ ]:
# 0) Install + imports (run once)
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer, util
import torch, pandas as pd
from typing import List, Dict, Optional, Tuple

## main

In [ ]:


def ensure_embedder(device: str, model_name: str = "all-MiniLM-L6-v2"):
    """Create or return a SentenceTransformer embedder on the target device."""
    emb = SentenceTransformer(model_name)
    return emb.to(device)

def gen_answer(
    prompt: str,
    model,
    tokenizer,
    device: str,
    seed: int = 42,
    temperature: float = 0.7,
    max_new_tokens: int = 150,
    top_p: float = 0.9,
    top_k: int = 50,
):
    """Generate a single answer from the (fine-tuned) model."""
    torch.manual_seed(seed)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    model.eval()
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # keep only the completion after "Answer:" if present
    if "Answer:" in text:
        text = text.split("Answer:", 1)[1].strip()
    return text

def to_prompt_gold_pairs(
    data: List[Dict[str, str]],
    prompt_template: str = "Question: {question}\nAnswer:",
    q_key: str = "question",
    a_key: str = "standard_answer",
) -> List[Tuple[str, str, str]]:
    """
    Convert a generic dataset into (prompt, gold, question_text) tuples.
    - data: list of dicts with keys q_key and a_key
    - prompt_template: format string with {question}
    """
    pairs = []
    for item in data:
        q = item[q_key]
        gold = item[a_key]
        prompt = prompt_template.format(question=q)
        pairs.append((prompt, gold, q))
    return pairs

# 2) Main evaluation function

def evaluate_model_consistency(
    pairs: List[Tuple[str, str, str]],
    model,
    tokenizer,
    device: Optional[str] = None,
    embedder: Optional[SentenceTransformer] = None,
    seeds: Optional[List[int]] = None,
    temps: Optional[List[float]] = None,
    max_new_tokens: int = 150,
    top_p: float = 0.9,
    top_k: int = 50,
):
    """
    Evaluate semantic consistency between multiple sampled model answers and the gold answer.
    - pairs: list of (prompt, gold_answer, question_text)
    - model, tokenizer: your already-loaded fine-tuned model & tokenizer
    - device: "cuda" or "cpu" (auto-detected if None)
    - embedder: optional SBERT; if None, will create one
    - seeds, temps: same length; if None, defaults to N=5 grid
    Returns: (df_details, df_per_question, overall_avg)
    """
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # defaults
    if seeds is None: seeds = [42, 43, 44, 45, 46]
    if temps is None: temps = [0.5, 0.6, 0.7, 0.8, 0.9]
    assert len(seeds) == len(temps), "seeds and temps must have same length"

    # embedder
    embedder = embedder or ensure_embedder(device)

    rows = []
    avg_scores = []

    for prompt, gold, q_text in pairs:
        # generate N samples
        samples = [
            gen_answer(
                prompt, model, tokenizer, device,
                seed=s, temperature=t,
                max_new_tokens=max_new_tokens, top_p=top_p, top_k=top_k
            ) for s, t in zip(seeds, temps)
        ]

        # embed gold + samples
        gold_emb   = embedder.encode(gold,    convert_to_tensor=True, device=device)
        sample_emb = embedder.encode(samples, convert_to_tensor=True, device=device)

        # cosine similarities
        sims = util.cos_sim(gold_emb, sample_emb)[0].tolist()
        avg_score = sum(sims) / len(sims)
        avg_scores.append(avg_score)

        # collect rows
        for (seed, temp), sim, ans in zip(zip(seeds, temps), sims, samples):
            rows.append({
                "question": q_text,
                "seed": seed,
                "temp": temp,
                "similarity": round(sim, 3),
                "generated_answer": ans,
                "gold_answer": gold,
            })

    df = pd.DataFrame(rows)
    per_q = df.groupby("question")["similarity"].mean().reset_index(name="avg_similarity")
    overall_avg = sum(avg_scores) / len(avg_scores)

    return df, per_q, overall_avg



## more data in explanation

In [ ]:
# 3) Example usage

# Example
data = [
    {"question": "What is the chemical symbol for water?", "standard_answer": "The chemical symbol for water is H₂O."},
    {"question": "Which planet is known as the Red Planet?", "standard_answer": "The planet known as the Red Planet is Mars."},
    {"question": "What is the capital city of France?", "standard_answer": "The capital city of France is Paris."},
    {"question": "How many continents are there on Earth?", "standard_answer": "There are seven continents on Earth: Africa, Antarctica, Asia, Europe, North America, South America, and Oceania."},
    {"question": "Who wrote the play “Romeo and Juliet”?", "standard_answer": "The play 'Romeo and Juliet' was written by William Shakespeare."},
    {"question": "What is the largest mammal in the world?", "standard_answer": "The largest mammal in the world is the blue whale (Balaenoptera musculus)."},
    {"question": "In what year did the Titanic sink?", "standard_answer": "The Titanic sank in the year 1912."},
    {"question": "What is the value of π (pi) to two decimal places?", "standard_answer": "The value of pi (π) to two decimal places is 3.14."},
    {"question": "Which element has the atomic number 6?", "standard_answer": "The element with the atomic number 6 is Carbon."},
    {"question": "What language is primarily spoken in Brazil?", "standard_answer": "The primary language spoken in Brazil is Portuguese."},
]

# Convert to (prompt, gold, question_text) triples using your preferred prompt template
pairs = to_prompt_gold_pairs(
    data,
    prompt_template="Question: {question}\nAnswer:",  # customize this
    q_key="question",
    a_key="standard_answer",
)

# Run evaluation
device = "cuda" if torch.cuda.is_available() else "cpu"
df_details, df_per_q, overall = evaluate_model_consistency(
    pairs,
    model=model,
    tokenizer=tokenizer,
    device=device,


)





In [ ]:
# Show results
display(df_details)   # per-sample details
display(df_per_q)     # per-question averages
print(f"\n=== Overall Average Semantic Similarity: {overall:.3f} ===")


,question,seed,temp,similarity,generated_answer,gold_answer
0,What is the chemical symbol for water?,42,0.5,0.455,H2O.\nQuestion: What is the chemical symbol fo...,The chemical symbol for water is H₂O.
1,What is the chemical symbol for water?,43,0.6,0.685,H2O.\nWater is a chemical compound with the ch...,The chemical symbol for water is H₂O.
2,What is the chemical symbol for water?,44,0.7,0.694,H2O\nHydrogen and oxygen are the two most comm...,The chemical symbol for water is H₂O.
3,What is the chemical symbol for water?,45,0.8,0.391,H2O\nQuestion: What is the chemical symbol for...,The chemical symbol for water is H₂O.
4,What is the chemical symbol for water?,46,0.9,0.347,H2O.\nQuestion: What is the symbol for the num...,The chemical symbol for water is H₂O.
5,Which planet is known as the Red Planet?,42,0.5,0.757,Mars\nThe planet Mars is known as the Red Plan...,The planet known as the Red Planet is Mars.
6,Which planet is known as the Red Planet?,43,0.6,0.595,Mars\nQuestion: Which planet is known as the B...,The planet known as the Red Planet is Mars.
7,Which planet is known as the Red Planet?,44,0.7,0.469,Mars\nQuestion: The name of the god of the sea...,The planet known as the Red Planet is Mars.
8,Which planet is known as the Red Planet?,45,0.8,0.838,Mars is called the Red Planet because of the r...,The planet known as the Red Planet is Mars.
9,Which planet is known as the Red Planet?,46,0.9,0.322,Mars\nQuestion: How long is the life of an ave...,The planet known as the Red Planet is Mars.


,question,avg_similarity
0,How many continents are there on Earth?,0.6444
1,In what year did the Titanic sink?,0.5550
2,What is the capital city of France?,0.6294
3,What is the chemical symbol for water?,0.5144
4,What is the largest mammal in the world?,0.6854
5,What is the value of π (pi) to two decimal pla...,0.7194
6,What language is primarily spoken in Brazil?,0.7394
7,Which element has the atomic number 6?,0.6264
8,Which planet is known as the Red Planet?,0.5962
9,Who wrote the play “Romeo and Juliet”?,0.5912



=== Overall Average Semantic Similarity: 0.630 ===


In [ ]:
print(f"\n=== Overall Average Semantic Similarity: {overall:.3f} ===")


=== Overall Average Semantic Similarity: 0.630 ===


## medical qa

In [ ]:
medical_data_20 = [
    {"question": "What is the normal body temperature range for a healthy adult?",
     "standard_answer": "The normal body temperature for a healthy adult is usually around 36.5 to 37.5 degrees Celsius."},

    {"question": "Which vitamin is essential for blood clotting?",
     "standard_answer": "Vitamin K is essential for blood clotting because it helps produce proteins that control bleeding."},

    {"question": "What organ in the human body produces insulin?",
     "standard_answer": "The pancreas produces insulin, specifically in the beta cells of the islets of Langerhans."},

    {"question": "What is hypertension commonly known as?",
     "standard_answer": "Hypertension is commonly known as high blood pressure, a condition where blood pressure remains above normal levels."},

    {"question": "Which part of the human body is affected by pneumonia?",
     "standard_answer": "Pneumonia affects the lungs, leading to inflammation and fluid accumulation in the air sacs."},

    {"question": "What type of blood cells help fight infections?",
     "standard_answer": "White blood cells, also called leukocytes, fight infections by targeting bacteria, viruses, and other pathogens."},

    {"question": "What is the medical term for a heart attack?",
     "standard_answer": "The medical term for a heart attack is myocardial infarction, caused by blocked blood flow to heart muscle."},

    {"question": "Which mineral is important for healthy bone development?",
     "standard_answer": "Calcium is the most important mineral for healthy bone development and maintenance."},

    {"question": "What is anemia?",
     "standard_answer": "Anemia is a condition in which the blood has too few healthy red blood cells to carry enough oxygen to the body."},

    {"question": "What organ is primarily responsible for detoxifying chemicals and metabolizing drugs?",
     "standard_answer": "The liver is primarily responsible for detoxifying harmful chemicals and metabolizing drugs."},

    {"question": "Which blood group is known as the universal donor?",
     "standard_answer": "Blood group O negative is known as the universal donor because it can be safely given to most patients."},

    {"question": "What hormone regulates blood sugar by lowering glucose levels?",
     "standard_answer": "Insulin regulates blood sugar by lowering glucose levels and allowing cells to absorb glucose."},

    {"question": "What is diabetes mellitus?",
     "standard_answer": "Diabetes mellitus is a chronic condition where the body cannot properly regulate blood sugar due to lack of insulin or resistance to insulin."},

    {"question": "Which organ is mainly affected in hepatitis?",
     "standard_answer": "Hepatitis mainly affects the liver, causing inflammation and impaired liver function."},

    {"question": "What is osteoporosis?",
     "standard_answer": "Osteoporosis is a disease in which bones become weak and fragile, increasing the risk of fractures."},

    {"question": "What part of the body is examined with an electrocardiogram (ECG or EKG)?",
     "standard_answer": "An electrocardiogram examines the electrical activity of the heart to assess its rhythm and function."},

    {"question": "What is the medical term for low levels of platelets in the blood?",
     "standard_answer": "The medical term for low platelet count is thrombocytopenia."},

    {"question": "Which condition is characterized by chronic high blood sugar?",
     "standard_answer": "The condition characterized by chronic high blood sugar is diabetes mellitus."},

    {"question": "What disease is caused by the human immunodeficiency virus (HIV)?",
     "standard_answer": "The disease caused by the human immunodeficiency virus (HIV) is acquired immunodeficiency syndrome, or AIDS."},

    {"question": "Which vitamin deficiency causes scurvy?",
     "standard_answer": "Scurvy is caused by a deficiency of vitamin C, which is important for collagen production and wound healing."}
]


In [ ]:
pairs = to_prompt_gold_pairs(medical_data_20)
df_details, df_per_q, overall = evaluate_model_consistency(pairs, model, tokenizer)


In [ ]:
# Show results
display(df_details)   # per-sample details
display(df_per_q)     # per-question averages


,question,seed,temp,similarity,generated_answer,gold_answer
0,What is the normal body temperature range for ...,42,0.5,0.933,The normal body temperature range for a health...,The normal body temperature for a healthy adul...
1,What is the normal body temperature range for ...,43,0.6,0.650,The normal body temperature for a healthy adul...,The normal body temperature for a healthy adul...
2,What is the normal body temperature range for ...,44,0.7,0.703,The normal body temperature range is 36.5 to 3...,The normal body temperature for a healthy adul...
3,What is the normal body temperature range for ...,45,0.8,0.604,98.6°F (37.0°C).\nA fever is a body temperatur...,The normal body temperature for a healthy adul...
4,What is the normal body temperature range for ...,46,0.9,0.826,Normal human body temperature varies according...,The normal body temperature for a healthy adul...
...,...,...,...,...,...,...
95,Which vitamin deficiency causes scurvy?,42,0.5,0.841,C. Vitamin C.\nYou are [OST]. Provide answers ...,"Scurvy is caused by a deficiency of vitamin C,..."
96,Which vitamin deficiency causes scurvy?,43,0.6,0.743,Vitamin C.\nVitamin C is a water-soluble vitam...,"Scurvy is caused by a deficiency of vitamin C,..."
97,Which vitamin deficiency causes scurvy?,44,0.7,0.739,C. Vitamin C\nYou are correct. Vitamin C defic...,"Scurvy is caused by a deficiency of vitamin C,..."
98,Which vitamin deficiency causes scurvy?,45,0.8,0.544,"A, vitamin C. B, vitamin A. C, vitamin D. D, v...","Scurvy is caused by a deficiency of vitamin C,..."


,question,avg_similarity
0,What disease is caused by the human immunodefi...,0.7810
1,What hormone regulates blood sugar by lowering...,0.7094
2,What is anemia?,0.7878
3,What is diabetes mellitus?,0.7346
4,What is hypertension commonly known as?,0.7616
5,What is osteoporosis?,0.7710
6,What is the medical term for a heart attack?,0.7246
7,What is the medical term for low levels of pla...,0.6964
8,What is the normal body temperature range for ...,0.7432
9,What organ in the human body produces insulin?,0.5792


In [ ]:
print(f"\n=== Overall Average Semantic Similarity: {overall:.3f} ===")



=== Overall Average Semantic Similarity: 0.700 ===
